In [106]:
import pandas as pd
import numpy as np
import os
import io
from google.cloud import storage
from pandas_gbq import to_gbq
import glob

In [107]:
storage_client = storage.Client()
bucket_name = 'mh_simpsons'
bucket = storage_client.get_bucket(bucket_name)

dfs = {}

for blob in bucket.list_blobs():
    if blob.name.endswith('.csv'):
        blob_content = blob.download_as_string().decode('utf-8')
        df = pd.read_csv(io.StringIO(blob_content), dtype=str)
        df_name = blob.name.split('/')[-1].split('.')[0]
        dfs[df_name] = df

In [108]:
dfs.keys()

dict_keys(['simpsons_characters', 'simpsons_episodes', 'simpsons_locations', 'simpsons_script_lines'])

In [109]:
dfs['simpsons_characters']

,id,name,normalized_name,gender
0,7,Children,children,NaN
1,12,Mechanical Santa,mechanical santa,NaN
2,13,Tattoo Man,tattoo man,NaN
3,16,DOCTOR ZITSOFSKY,doctor zitsofsky,NaN
4,20,Students,students,NaN
...,...,...,...,...
6717,5222,Ron Rabinowitz,ron rabinowitz,m
6718,5728,Martha Stewart,martha stewart,f
6719,1770,Officer Goodman,officer goodman,m
6720,1634,Evan Conover,evan conover,m


In [110]:
dfs['simpsons_characters'].head()
dfs['simpsons_characters'].isnull().sum(axis = 0)

id                    0
name                  0
normalized_name       0
gender             6399
dtype: int64

In [111]:
def get_na(df):
    qsna = df.shape[0] - df.isnull().sum(axis=0)
    qna = df.isnull().sum(axis=0)
    ppna = round(100 * (qna / df.shape[0]), 2)
    aux = {'Datos sin Nas en qtd': qsna, 'Na en qtd': qna, 'Na en %': ppna}
    na = pd.DataFrame(data=aux)
    return na.sort_values(by='Na en qtd', ascending=False)

In [112]:
get_na(dfs['simpsons_characters'])

,Datos sin Nas en qtd,Na en qtd,Na en %
gender,323,6399,95.19
id,6722,0,0.00
name,6722,0,0.00
normalized_name,6722,0,0.00


In [113]:
dfs['simpsons_characters'].head()

,id,name,normalized_name,gender
0,7,Children,children,NaN
1,12,Mechanical Santa,mechanical santa,NaN
2,13,Tattoo Man,tattoo man,NaN
3,16,DOCTOR ZITSOFSKY,doctor zitsofsky,NaN
4,20,Students,students,NaN


In [114]:
dfs['simpsons_characters'] = dfs['simpsons_characters'].drop(columns=['gender','name'], axis=1)
dfs['simpsons_characters'].head(3)

,id,normalized_name
0,7,children
1,12,mechanical santa
2,13,tattoo man


In [115]:
df1_con_nulos = dfs['simpsons_characters'][dfs['simpsons_characters'].isnull().any(axis=1)]
df1_con_nulos.head(6)
df1_con_nulos

,id,normalized_name


In [116]:
dfs['simpsons_episodes'].head()
dfs['simpsons_episodes'].isnull().sum(axis = 0)

id                        0
image_url                 4
imdb_rating               3
imdb_votes                3
number_in_season          0
number_in_series          0
original_air_date         0
original_air_year         0
production_code           0
season                    0
title                     0
us_viewers_in_millions    6
video_url                 4
views                     4
dtype: int64

In [117]:
get_na(dfs['simpsons_episodes'])

,Datos sin Nas en qtd,Na en qtd,Na en %
us_viewers_in_millions,594,6,1.00
image_url,596,4,0.67
video_url,596,4,0.67
views,596,4,0.67
imdb_rating,597,3,0.50
imdb_votes,597,3,0.50
id,600,0,0.00
number_in_season,600,0,0.00
number_in_series,600,0,0.00
original_air_date,600,0,0.00


In [118]:
df2_con_nulos = dfs['simpsons_episodes'][dfs['simpsons_episodes'].isnull().any(axis=1)]
df2_con_nulos

,id,image_url,imdb_rating,imdb_votes,number_in_season,number_in_series,original_air_date,original_air_year,production_code,season,title,us_viewers_in_millions,video_url,views
59,160,http://static-media.fxx.com/img/FX_Networks_-_...,7.8,1005.0,7,160,1996-12-15,1996,4F01,8,Lisa's Date with Density,NaN,http://www.simpsonsworld.com/video/306394691862,60912.0
65,173,http://static-media.fxx.com/img/FX_Networks_-_...,7.7,913.0,20,173,1997-04-13,1997,4F16,8,The Canine Mutiny,NaN,http://www.simpsonsworld.com/video/436370499586,54155.0
189,447,NaN,6.7,543.0,6,447,2009-11-22,2009,LABF18,21,Pranks and Greens,7.03,NaN,NaN
234,598,NaN,NaN,NaN,2,598,2016-10-02,2016,VABF18,28,"Friends and Family""[203]",NaN,NaN,NaN
235,599,NaN,NaN,NaN,3,599,2016-10-09,2016,VABF17,28,"The Town""[205]",NaN,NaN,NaN
236,600,NaN,NaN,NaN,4,600,2016-10-16,2016,VABF16,28,"Treehouse of Horror XXVII""[207]",NaN,NaN,NaN
320,161,http://static-media.fxx.com/img/FX_Networks_-_...,8.8,1268.0,8,161,1996-12-29,1996,4F07,8,Hurricane Neddy,NaN,http://www.simpsonsworld.com/video/436250179702,66281.0


In [119]:
columns = ['us_viewers_in_millions', 'views', 'imdb_rating', 'imdb_votes']
dfs['simpsons_episodes'][columns] = dfs['simpsons_episodes'][columns].bfill()

In [120]:
get_na(dfs['simpsons_episodes'])

,Datos sin Nas en qtd,Na en qtd,Na en %
image_url,596,4,0.67
video_url,596,4,0.67
id,600,0,0.00
imdb_rating,600,0,0.00
imdb_votes,600,0,0.00
number_in_season,600,0,0.00
number_in_series,600,0,0.00
original_air_date,600,0,0.00
original_air_year,600,0,0.00
production_code,600,0,0.00


In [121]:
dfs['simpsons_locations'].head()

,id,name,normalized_name
0,1,Street,street
1,2,Car,car
2,3,Springfield Elementary School,springfield elementary school
3,4,Auditorium,auditorium
4,5,Simpson Home,simpson home


In [122]:
dfs['simpsons_locations'].isnull().sum(axis = 0)

id                 0
name               0
normalized_name    0
dtype: int64

In [123]:
dfs['simpsons_locations'] = dfs['simpsons_locations'].drop(columns= 'name' , axis=1)

In [124]:
get_na(dfs['simpsons_locations'])

,Datos sin Nas en qtd,Na en qtd,Na en %
id,4459,0,0.0
normalized_name,4459,0,0.0


In [125]:
dfs['simpsons_script_lines'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158271 entries, 0 to 158270
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id                  158271 non-null  object
 1   episode_id          158271 non-null  object
 2   number              158271 non-null  object
 3   raw_text            158271 non-null  object
 4   timestamp_in_ms     158271 non-null  object
 5   speaking_line       158271 non-null  object
 6   character_id        140750 non-null  object
 7   location_id         157864 non-null  object
 8   raw_character_text  140749 non-null  object
 9   raw_location_text   157863 non-null  object
 10  spoken_words        132112 non-null  object
 11  normalized_text     132087 non-null  object
 12  word_count          132112 non-null  object
dtypes: object(13)
memory usage: 15.7+ MB


In [126]:
dfs['simpsons_script_lines'].isnull().sum(axis = 0)

id                        0
episode_id                0
number                    0
raw_text                  0
timestamp_in_ms           0
speaking_line             0
character_id          17521
location_id             407
raw_character_text    17522
raw_location_text       408
spoken_words          26159
normalized_text       26184
word_count            26159
dtype: int64

In [127]:
get_na(dfs['simpsons_script_lines'])

,Datos sin Nas en qtd,Na en qtd,Na en %
normalized_text,132087,26184,16.54
spoken_words,132112,26159,16.53
word_count,132112,26159,16.53
raw_character_text,140749,17522,11.07
character_id,140750,17521,11.07
raw_location_text,157863,408,0.26
location_id,157864,407,0.26
id,158271,0,0.00
episode_id,158271,0,0.00
number,158271,0,0.00


In [128]:
dfs['simpsons_script_lines'].head(3)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,true,464,3,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,true,9,3,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,true,464,3,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22


In [129]:
dfs['simpsons_script_lines'] = dfs['simpsons_script_lines'].drop(columns= 'normalized_text' , axis=1)

In [130]:
columns= ['spoken_words', 'spoken_words', 'raw_location_text', 'raw_character_text', 'raw_text']

In [131]:
columns = ['spoken_words', 'raw_location_text', 'raw_character_text', 'raw_text']
dfs['simpsons_script_lines'][columns] = dfs['simpsons_script_lines'][columns].astype(str).replace('', np.nan).fillna('').apply(lambda x: x.str.title())


In [132]:
dfs['simpsons_script_lines'].head(3)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,word_count
0,9549,32,209,"Miss Hoover: No, Actually, It Was A Little Of ...",848000,true,464,3,Miss Hoover,Springfield Elementary School,"No, Actually, It Was A Little Of Both. Sometim...",31
1,9550,32,210,Lisa Simpson: (Near Tears) Where'S Mr. Bergstrom?,856000,true,9,3,Lisa Simpson,Springfield Elementary School,Where'S Mr. Bergstrom?,3
2,9551,32,211,Miss Hoover: I Don'T Know. Although I'D Sure L...,856000,true,464,3,Miss Hoover,Springfield Elementary School,I Don'T Know. Although I'D Sure Like To Talk T...,22


In [133]:
project_id = 'massive-haiku-422112-q2'
dataset = 'simpsons_ds'

for df_name, df in dfs.items():
    table_name = df_name.replace('df', '')
    destination_table = f'{project_id}.{dataset}.{table_name}'
    to_gbq(df, destination_table, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 6384.02it/s]
